In [1]:
import os
import dotenv
import inspect
dotenv.load_dotenv()

True

In [2]:
from langchain import OpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory, 
                                                  ConversationSummaryMemory, 
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.callbacks import get_openai_callback
import tiktoken

In [19]:
llm = OpenAI(temperature=0,
             openai_api_base=os.getenv('OPENAI_REVERSE_PROXY'),
             openai_api_key=os.getenv('OPENAI_API_KEY')
)

In [20]:
#before delving into the different memory modules that the library offers, we will introduce the chain will be using for these examples

conversation =  ConversationChain(
    llm=llm
)

In [21]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [22]:
#Take a look at what the chain is doing with this prompt

print(inspect.getsource(conversation._call), inspect.getsource(conversation.apply))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        response = self.generate([inputs], run_manager=run_manager)
        return self.create_outputs(response)[0]
     def apply(
        self, input_list: List[Dict[str, Any]], callbacks: Callbacks = None
    ) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        callback_manager = CallbackManager.configure(
            callbacks, self.callbacks, self.verbose
        )
        run_manager = callback_manager.on_chain_start(
            dumpd(self),
            {"input_list": input_list},
        )
        try:
            response = self.generate(input_list, run_manager=run_manager)
        except (KeyboardInterrupt, Exception) as e:
            run_manager.on_chain_error(e)
            raise e
        outputs = self.create_outputs(response)
        run_manager.on_chain_end({"outputs": output

In [23]:
#If we compare it with LLMChain

print(inspect.getsource(LLMChain._call), inspect.getsource(LLMChain.apply))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        response = self.generate([inputs], run_manager=run_manager)
        return self.create_outputs(response)[0]
     def apply(
        self, input_list: List[Dict[str, Any]], callbacks: Callbacks = None
    ) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        callback_manager = CallbackManager.configure(
            callbacks, self.callbacks, self.verbose
        )
        run_manager = callback_manager.on_chain_start(
            dumpd(self),
            {"input_list": input_list},
        )
        try:
            response = self.generate(input_list, run_manager=run_manager)
        except (KeyboardInterrupt, Exception) as e:
            run_manager.on_chain_error(e)
            raise e
        outputs = self.create_outputs(response)
        run_manager.on_chain_end({"outputs": output

## Memory types

In [24]:
# ConversationBufferMemory

# The ConversationBufferMemory does just what its name suggests: it keeps a buffer of the previous conversation excerpts as part of the context in the prompt

In [29]:
# Create a token countere

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')
    return result

In [30]:
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [31]:
conversation_buf("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': " Good morning! It's a beautiful day today, isn't it? How can I help you?"}

In [32]:
count_tokens(
    conversation_buf, 
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 178 tokens


' Interesting! Large Language Models are a type of artificial intelligence that can process natural language and generate text. They can be used to generate text from a given context, or to answer questions about a given context. Integrating them with external knowledge can help them to better understand the context and generate more accurate results. Do you have any specific questions about this integration?'

In [33]:
count_tokens(
    conversation_buf, 
    "Which data source types could be used to give context to the model?"
)

Spent a total of 286 tokens


' Generally, any type of data source can be used to give context to a Large Language Model. This could include text, images, audio, video, and other types of data. Depending on the type of data, different techniques may be used to integrate it with the model. For example, text data could be used to provide context through word embeddings, while images could be used to provide context through convolutional neural networks.'

In [34]:
count_tokens(conversation_buf, "What is my aim again?")

Spent a total of 315 tokens


' Your aim is to explore the potential of integrating Large Language Models with external knowledge.'

In [37]:
#clearly it remembers the past conversations

print(conversation_buf.memory.buffer)

Human: Good morning AI!
AI:  Good morning! It's a beautiful day today, isn't it? How can I help you?
Human: My interest here is to explore the potential of integrating Large Language Models with external knowledge
AI:  Interesting! Large Language Models are a type of artificial intelligence that can process natural language and generate text. They can be used to generate text from a given context, or to answer questions about a given context. Integrating them with external knowledge can help them to better understand the context and generate more accurate results. Do you have any specific questions about this integration?
Human: Which data source types could be used to give context to the model?
AI:  Generally, any type of data source can be used to give context to a Large Language Model. This could include text, images, audio, video, and other types of data. Depending on the type of data, different techniques may be used to integrate it with the model. For example, text data could be 

## Memory type 2: ConversationSummaryMemory

In [38]:
# The con of the conversation buffer memory is that it keeps the past conversations and stores it up hence takes away unnecessary tokens.

# Con...summm memory summarizes the text using llms and uses the summary of the past conversaions to render an output


conversation_sum  = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=llm)
)

In [39]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [40]:
#lets rerun the conversation again

count_tokens(
    conversation_sum,
    "Good Morning AI!"
)

Spent a total of 290 tokens


" Good morning! It's a beautiful day today, isn't it? How can I help you?"

In [41]:
count_tokens(
    conversation_sum,
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 440 tokens


" That sounds like an interesting project! I'm familiar with Large Language Models, but I'm not sure how they could be integrated with external knowledge. Could you tell me more about what you have in mind?"

In [42]:
count_tokens(
    conversation_sum, 
    "I just want to analyze the different possibilities. What can you think of?"
)


Spent a total of 664 tokens


' I can think of a few possibilities. One option is to use a large language model to generate a set of candidate answers to a given query, and then use external knowledge to filter out the most relevant answers. Another option is to use the large language model to generate a set of candidate answers, and then use external knowledge to score and rank the answers. Finally, you could use the large language model to generate a set of candidate answers, and then use external knowledge to refine the answers.'

In [43]:


count_tokens(
    conversation_sum, 
    "What is my aim again?"
)



Spent a total of 621 tokens


' Your aim is to explore the potential of integrating Large Language Models with external knowledge.'

In [45]:
print(conversation_sum.memory.buffer)

# Nice summary of the conversation


The human greeted the AI with a good morning, to which the AI responded with a good morning and asked how it could help. The human expressed interest in exploring the potential of integrating Large Language Models with external knowledge, to which the AI responded positively and asked for more information. The human asked the AI to think of different possibilities, and the AI suggested three options: using the large language model to generate a set of candidate answers and then using external knowledge to filter out the most relevant answers, score and rank the answers, or refine the answers. When the human asked what their aim was again, the AI reminded them that their aim was to explore the potential of integrating Large Language Models with external knowledge.


In [46]:
# we can count the number of tokens being used (wihtout making a call to OpenAI) using the tiktoken tokenizer


# initialize tokenizer
tokenizer = tiktoken.encoding_for_model('text-davinci-003')

# show number of tokens for the memory used by each memory type
print(
    f'Buffer memory conversation length: {len(tokenizer.encode(conversation_buf.memory.buffer))}\n'
    f'Summary memory conversation length: {len(tokenizer.encode(conversation_sum.memory.buffer))}'
)

Buffer memory conversation length: 261
Summary memory conversation length: 141


## Memory type3: ConversationBufferWindowMemory

In [58]:
#kind of short term memory. Only keeping the latest ones while omitting the older ones


conversation_bufw = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=3)
)

In [59]:


count_tokens(
    conversation_bufw, 
    "Good morning AI!"
)



Spent a total of 85 tokens


" Good morning! It's a beautiful day today, isn't it? How can I help you?"

In [60]:


count_tokens(
    conversation_bufw, 
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)



Spent a total of 178 tokens


' Interesting! Large Language Models are a type of artificial intelligence that can process natural language and generate text. They can be used to generate text from a given context, or to answer questions about a given context. Integrating them with external knowledge can help them to better understand the context and generate more accurate results. Do you have any specific questions about this integration?'

In [61]:
count_tokens(
    conversation_bufw, 
    "I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 264 tokens


' There are many possibilities for integrating Large Language Models with external knowledge. For example, you could use external knowledge to provide additional context to the model, or to provide additional training data. You could also use external knowledge to help the model better understand the context of a given text, or to help it generate more accurate results.'

In [62]:
count_tokens(
    conversation_bufw, 
    "Which data source types could be used to give context to the model?"
)


Spent a total of 344 tokens


'  Data sources that could be used to give context to the model include text corpora, images, audio, and video. Text corpora can provide additional context to the model by providing additional training data. Images, audio, and video can provide additional context by providing additional visual or auditory information.'

In [63]:


count_tokens(
    conversation_bufw, 
    "What is my aim again?"
)



Spent a total of 341 tokens


' Your aim is to explore the potential of integrating Large Language Models with external knowledge.'

In [64]:
#Essentially it forgot what my aim was, instead it made up things
# Since k=1 it remembers the last single conversation


bufw_history = conversation_bufw.memory.load_memory_variables(inputs=[])['history']

In [65]:
print(bufw_history)

Human: I just want to analyze the different possibilities. What can you think of?
AI:  There are many possibilities for integrating Large Language Models with external knowledge. For example, you could use external knowledge to provide additional context to the model, or to provide additional training data. You could also use external knowledge to help the model better understand the context of a given text, or to help it generate more accurate results.
Human: Which data source types could be used to give context to the model?
AI:   Data sources that could be used to give context to the model include text corpora, images, audio, and video. Text corpora can provide additional context to the model by providing additional training data. Images, audio, and video can provide additional context by providing additional visual or auditory information.
Human: What is my aim again?
AI:  Your aim is to explore the potential of integrating Large Language Models with external knowledge.


In [66]:
print(
    f'Buffer memory conversation length: {len(tokenizer.encode(conversation_buf.memory.buffer))}\n'
    f'Summary memory conversation length: {len(tokenizer.encode(conversation_sum.memory.buffer))}\n'
    f'Buffer window memory conversation length: {len(tokenizer.encode(bufw_history))}'
)

Buffer memory conversation length: 0
Summary memory conversation length: 141
Buffer window memory conversation length: 193


## Another type of memory is ConversationKnowledgeGraphMemory

In [68]:
!pip install -qU networkx

In [69]:
# This super cool memory type is based on the concept of a knowlede graph which recognizes different entities and connects them in pairs with a predicate


conversation_kg = ConversationChain(
    llm = llm,
    memory = ConversationKGMemory(llm=llm)
)

In [70]:
count_tokens(
    conversation_kg, 
    "My name is human and I like mangoes!"
)

Spent a total of 1168 tokens


" Hi Human! My name is AI. It's nice to meet you. I like mangoes too! Did you know that mangoes are a great source of vitamins A and C?"

In [71]:
conversation_kg.memory.kg.get_triples()

[('Human', 'human', 'name is'), ('Human', 'mangoes', 'likes')]